# Controllers

To actively manipulate the computing or energy systems during experiments, we need to implement a custom `Controller`.

In this example, our controller will adjust the power consumption of the compute nodes depending on the current power delta. 

In [1]:
from __future__ import annotations
import vessim as vs

# Hotfix to execute asyncio in Jupyter
import nest_asyncio
nest_asyncio.apply()

The most important aspect of this procedure is the implementation of the
`step()` method, which needs to be implemented as dictated by the Controller
ABC. The `step()` method is called every simulation step and allows a Controller
to act on the power delta from the Microgrid, the current simulation time and
the activity of the Actors. The `SimpleLoadBalancingController` only utilizes
the power delta and adjusts, depending on this value, the power consumption of
the computing system and in turn the power delta in the next simulation step.

In [2]:
class SimpleLoadBalancingController(vs.Controller):
    def __init__(self, nodes: list[vs.ConstantSignal]):
        super().__init__()
        self.nodes = nodes
        # save original node power consumption
        self.node_p = {node.name: node.now() for node in self.nodes}

    def step(self, time: int, p_delta: float, e_delta: float, state: dict) -> None:
        for node in self.nodes:
            new_power = self.node_p[node.name]
            if p_delta < 0:
                new_power *= 0.3
            node.set_value(new_power)

Now we can add the controller to the basic scenario by instantiating it with a
reference to the power meters it can control and the maximum load adjustment for
each step. The rest of the scenario remains unchanged.

In [3]:
environment = vs.Environment(sim_start="2022-06-09 00:00:00")

nodes: list = [
    vs.ConstantSignal(value=200),
    vs.ConstantSignal(value=250),
]
monitor = vs.Monitor()  # Stores simulation result on each step
load_balancer = SimpleLoadBalancingController(nodes=nodes)
environment.add_microgrid(
    actors=[
        vs.ComputingSystem(nodes=nodes, pue=1.6),
        vs.Actor(
            name="solar",
            signal=vs.Trace.load(
                "solcast2022_global", column="Berlin", params={"scale": 5000}
            ),
        ),
    ],
    storage=vs.SimpleBattery(capacity=500, initial_soc=0.8),
    controllers=[monitor, load_balancer],
    step_size=60,  # Global step size (can be overridden by actors or controllers)
)

environment.run(until=24 * 3600)  # 24h
monitor.to_csv("result.csv")

AttributeError: module 'vessim' has no attribute 'ComputingSystem'

Compared to the basic example, our computing system utilizes less power from the grid when solar energy production is inadequate, thanks to our load balancer.

In [ ]:
import pandas as pd
import plotly.graph_objects as go

df = pd.read_csv("result.csv", parse_dates=[0], index_col=0)
# divide e_delta by step size because e_delta is energy
df["grid_power"] = df["e_delta"] / 60

# Create the plot
fig = go.Figure()

# Add grid power trace
fig.add_trace(go.Scatter(
    x=df.index, 
    y=df["grid_power"], 
    mode='lines',
    name="Grid Power",
    line=dict(color="blue")
))

# Add solar power trace
fig.add_trace(go.Scatter(
    x=df.index, 
    y=df["solar.p"], 
    mode='lines',
    name="Solar",
    line=dict(color="orange")
))

# Update layout
fig.update_layout(
    title="Load Balancing Controller - Grid and Solar Power Over Time",
    xaxis_title="Time",
    yaxis_title="Power (W)",
    hovermode='x unified',
    showlegend=True,
    margin=dict(l=0, t=40, b=0, r=0)
)

# Update x-axis to show hours
fig.update_xaxes(tickformat="%H")

fig.show()